### Import libraries

In [ ]:
try:
    import pandas as pd
except ImportError:
    !pip install pandas
    import pandas as pd

try:
    import numpy as np
except ImportError:
    !pip install numpy
    import numpy as np

try:
    import kagglehub
except ImportError:
    !pip install kagglehub
    import kagglehub


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

### Load & Preprocess data

... continuar

In [ ]:
#import kagglehub
# Download latest version
#path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

#print("Path to dataset files:", path)


#(ESTÁ SUBIDO EN EL GITHUB)

In [ ]:
data_movies = pd.read_csv('datasets/movies-2/tmdb_5000_movies.csv')
data_credits = pd.read_csv('datasets/movies-2/tmdb_5000_credits.csv')

In [7]:
data_movies['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [25]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'; and replace NaN with an empty string
tfidf = TfidfVectorizer(stop_words='english')
data_movies['overview'] = data_movies['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(data_movies['overview'])
tfidf_matrix.shape

(4803, 20978)

In [ ]:
#Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [27]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(data_movies.index, index=data_movies['title']).drop_duplicates()

In [28]:
#Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    #Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    #Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    #Get the scores of the 10 most similar movies and their indices
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return data_movies['title'].iloc[movie_indices]

In [29]:
get_recommendations('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: title, dtype: object

In [30]:
get_recommendations('F.I.S.T.')

2918                  Jackass Number Two
3578                       House Party 2
3009                             Swimfan
3878    The Man Who Shot Liberty Valance
4048                         The Calling
1202                        Legal Eagles
4101                        Full Frontal
217                          Ghost Rider
2508                              Wasabi
3152                         Richard III
Name: title, dtype: object

In [31]:
get_recommendations('House Party 2')

2604                      Here On Earth
4625                   Midnight Cabaret
2246               Synecdoche, New York
3152                        Richard III
1256                             Ishtar
3079    Bucky Larson: Born to Be a Star
2334                            Birdman
3211                        Crazy Heart
3522                          Crossover
3185                             Gracie
Name: title, dtype: object